# Bike Sharing Demand：自転車利用数予測
https://www.kaggle.com/c/bike-sharing-demand
## Washington, D.C. の過去のレンタル自転車の利用パターン情報と気象データから、レンタル自転車の利用数を予測するモデルを作成する

### 評価指標：RMSLE
$$
\sqrt{\frac{1}{n}\sum_{i=0}^{n}(\log(p_i+1)-\log(a_i+1))^2}
$$
### data field
**datetime** - hourly date + timestamp  
**season** -  1 = spring, 2 = summer, 3 = fall, 4 = winter  
**holiday** - whether the day is considered a holiday  
**workingday** - whether the day is neither a weekend nor holiday  
**weather** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy  
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist  
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds  
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog  
**temp** - temperature in Celsius  
**atemp** - "feels like" temperature in Celsius  
**humidity** - relative humidity  
**windspeed** - wind speed  
**casual** - number of non-registered user rentals initiated  
**registered** - number of registered user rentals initiated  
**count** - number of total rentals  

In [32]:
#%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, ParameterGrid
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

### データの読み込み関連
ここら辺は特段難しいことはやってないので、基本コピペで問題ないです。  
C言語触ったことある人だったら、初見でもなんとなくやってることはわかるのではと思います。

In [33]:
TRAIN_DATA = 'input/train.csv'
TEST_DATA = 'input/test.csv'

parse_datesで指定した列をdatetime型で読み込んでおくと、後々便利

In [34]:
def read_csv(path):
    df = pd.read_csv(path, parse_dates=[0])
    return df

In [35]:
def load_train_data():
    df = read_csv(TRAIN_DATA)
    return df

In [36]:
def load_test_data():
    df = read_csv(TEST_DATA)
    return df

In [37]:
df = load_train_data()
test = load_test_data()

### データの確認

In [38]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [39]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [40]:
print(df.shape)
print(test.shape)

(10886, 12)
(6493, 9)


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null datetime64[ns]
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.6 KB


今回のデータは欠損値がありません。素晴らしい。

In [42]:
df.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

## feature engineering
datetimeを年月日、時間に分割

In [43]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [44]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['dayofweek'] = test['datetime'].dt.dayofweek
test['hour'] = test['datetime'].dt.hour

### 評価の際に、scikit-learnに用意されているrmse関数を使用するため、logに変換しておく

In [45]:
df['count'] = np.log(df['count'] + 1)
df.rename(columns={'count':'rentals'}, inplace=True)

In [46]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,rentals,year,month,day,dayofweek,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,2.833213,2011,1,1,5,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,3.713572,2011,1,1,5,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,3.496508,2011,1,1,5,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,2.639057,2011,1,1,5,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,0.693147,2011,1,1,5,4


### モデル作成
まずは何も考えず、ランダムフォレストでやってみる

In [47]:
removed_cols = ['rentals', 'casual', 'registered', 'datetime']
feats = [c for c in df.columns if c not in removed_cols]
x_train = df[feats]
y_train = df['rentals'].values

In [48]:
clf = RandomForestRegressor(random_state=0)
clf.fit(x_train, y_train)

C:\ProgramData\Anaconda3-5.1.0\envs\tensorflow-gpu-new\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

### 予測精度の評価

In [49]:
def logmse(y, pred):
    g = mean_squared_error(y, pred)**(1/2)
    return g

In [50]:
y_pred = clf.predict(x_train)
print(logmse(y_train, y_pred))

0.13520927717260978


あとはテストデータに対してもpredictして提出用ファイルを作成すればOKです。
提出用ファイルの作り方は割愛。  
githubに色々とコード載せているので、興味ある方は参照ください。  
https://github.com/msk0119/kaggle-bike/blob/master/xgb_cv5.py

### 評価方法
上で算出した予測精度は、学習データに対する評価スコア。  
実際には未知データに対して予測する必要があるため、学習データでテストしてもモデルの汎化能力は評価できない。  
そのため、学習データのうち、いくつかのデータを評価用データとする手法で評価を行うのが一般的。
#### ホールドアウト法

In [53]:
x_trn, x_val, y_trn, y_val = train_test_split(x_train, y_train, test_size=0.30, shuffle = False)#時系列データのためshuffleしない方がいいです。
clf = RandomForestRegressor(random_state=0)
clf.fit(x_trn, y_trn)
y_trn_pred = clf.predict(x_trn)
y_val_pred = clf.predict(x_val)
print("trainスコア", logmse(y_trn, y_trn_pred))
print("validスコア", logmse(y_val, y_val_pred))

C:\ProgramData\Anaconda3-5.1.0\envs\tensorflow-gpu-new\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


trainスコア 0.14537837404617743
validスコア 0.4051887685517535


#### クロスバリデーション

In [54]:
kf = KFold(n_splits=3, shuffle=False)

list_logmse_score = []

for train_idx, valid_idx in kf.split(x_train, y_train):
    trn_x = x_train.iloc[train_idx, :]
    val_x = x_train.iloc[valid_idx, :]
    
    trn_y = y_train[train_idx]
    val_y = y_train[valid_idx]
    
    clf = RandomForestRegressor(random_state=0)
    clf.fit(trn_x, trn_y)
    y_pred = clf.predict(val_x)
    
    sc_logmse = logmse(val_y, y_pred)
    
    list_logmse_score.append(sc_logmse)
    
sc_logmse = np.mean(list_logmse_score)
print(list_logmse_score)
print(sc_logmse)

C:\ProgramData\Anaconda3-5.1.0\envs\tensorflow-gpu-new\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3-5.1.0\envs\tensorflow-gpu-new\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3-5.1.0\envs\tensorflow-gpu-new\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.5857743973534578, 0.6310439471772865, 0.40602498851656715]
0.5409477776824372


以上で正しい評価スコアが算出できるため、評価スコアが向上するように、序盤はひたすらにfeature engineeringを頑張るべき。  
ある程度いい評価スコアを得られる変数を特定できれば、終盤はハイパーパラメータの探索、モデルのアンサンブル等を行う。

### ハイパーパラメータの探索

In [55]:
kf = KFold(n_splits=3, shuffle=False, random_state=0)

all_params = {'max_depth': [3, 5, 7],
              "n_estimators": [10, 100],
              'min_samples_split':[2, 4],
              'min_samples_leaf':[1, 3],
            'random_state':[0]}

min_score = 100
min_params = None

for params in tqdm(list(ParameterGrid(all_params))):
    
    list_logmse_score = []

    for train_idx, valid_idx in kf.split(x_train, y_train):
        trn_x = x_train.iloc[train_idx, :]
        val_x = x_train.iloc[valid_idx, :]
    
        trn_y = y_train[train_idx]
        val_y = y_train[valid_idx]
    
        clf = RandomForestRegressor(**params)
        clf.fit(trn_x, trn_y)
        y_pred = clf.predict(val_x)
    
        sc_logmse = logmse(val_y, y_pred)
    
        list_logmse_score.append(sc_logmse)

    sc_logmse = np.mean(list_logmse_score)
    if min_score > sc_logmse:
        min_score = sc_logmse
        min_params = params
        
print('minimam logmse:',min_score)

100%|██████████████████████████████████████████| 24/24 [00:22<00:00,  1.42s/it]


minimam logmse: 0.6369745272742983
